In [1]:
import pandas as pd
import numpy as np

In [14]:
d = pd.read_csv(r"C:\Users\ponra\Downloads\mobile_price_classification.csv")

In [15]:
d.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [16]:
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  2000 non-null   int64  
 1   blue           2000 non-null   int64  
 2   clock_speed    2000 non-null   float64
 3   dual_sim       2000 non-null   int64  
 4   fc             2000 non-null   int64  
 5   four_g         2000 non-null   int64  
 6   int_memory     2000 non-null   int64  
 7   m_dep          2000 non-null   float64
 8   mobile_wt      2000 non-null   int64  
 9   n_cores        1841 non-null   object 
 10  pc             2000 non-null   int64  
 11  px_height      2000 non-null   int64  
 12  px_width       2000 non-null   int64  
 13  ram            2000 non-null   int64  
 14  sc_h           2000 non-null   int64  
 15  sc_w           2000 non-null   int64  
 16  talk_time      2000 non-null   int64  
 17  three_g        2000 non-null   int64  
 18  touch_sc

In [17]:
d.drop('n_cores',axis=1,inplace=True)

In [18]:
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  2000 non-null   int64  
 1   blue           2000 non-null   int64  
 2   clock_speed    2000 non-null   float64
 3   dual_sim       2000 non-null   int64  
 4   fc             2000 non-null   int64  
 5   four_g         2000 non-null   int64  
 6   int_memory     2000 non-null   int64  
 7   m_dep          2000 non-null   float64
 8   mobile_wt      2000 non-null   int64  
 9   pc             2000 non-null   int64  
 10  px_height      2000 non-null   int64  
 11  px_width       2000 non-null   int64  
 12  ram            2000 non-null   int64  
 13  sc_h           2000 non-null   int64  
 14  sc_w           2000 non-null   int64  
 15  talk_time      2000 non-null   int64  
 16  three_g        2000 non-null   int64  
 17  touch_screen   2000 non-null   int64  
 18  wifi    

In [28]:
import warnings
warnings.filterwarnings("ignore")

# Bagging

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [30]:
x = d.iloc[:,:-1]
y = d.iloc[:,-1]

In [31]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25)

In [38]:
from sklearn.model_selection import GridSearchCV
parameters = [ 
                #{'penalty':['l1','l2','none']},
                {'max_depth':[5,6,7,8,9]},
                {'kernel':['sigmoid','linear']},
                {'var_smoothing':[0,1,2]},
                {'n_neighbors':[12,4,5,8,10,7]},
            ]

In [39]:
bags = 5
models = [DecisionTreeClassifier(),SVC(),GaussianNB(),KNeighborsClassifier()]

a = np.arange(x_train.shape[0])
np.random.seed(13)
for i in range(len(models)):
    np.random.shuffle(a)
    r = int(0.8*len(a))
    
    bagx = x_train.iloc[a[:r],:]
    bagy = y_train.iloc[a[:r]]

    classGrid = GridSearchCV(models[i],param_grid=parameters[i],cv=5,scoring='accuracy')
    classGrid.fit(bagx,bagy)
    
    print(models[i])
    print(classGrid.best_params_)
    print(classGrid.best_score_)
    
    models[i].set_params(**classGrid.best_params_)
    models[i].fit(bagx,bagy)

DecisionTreeClassifier()
{'max_depth': 8}
0.8116666666666668
SVC()
{'kernel': 'linear'}
0.96
GaussianNB()
{'var_smoothing': 0}
0.79
KNeighborsClassifier()
{'n_neighbors': 7}
0.9166666666666666


In [46]:
m=[]
for i in models:
    y_pred = i.predict(x_test)
    #print(accuracy_score(y_pred,y_test))
    m.append((accuracy_score(y_pred,y_test),i))

In [47]:
max(m)

(0.964, SVC(kernel='linear'))